In [45]:
import pandas as pd

prefix_path = 'D:/jeju_bus_data_no_leakage/bis/'
infix_path = 'BIS_' + '190531'
postfix_path = '.csv'

path = prefix_path + infix_path + postfix_path

df = pd.read_csv(path, 
                 usecols = ['EVENT_DATE', 'EVENT_CD', 'RUN_DIST',
                            'ROUTE_ID', 'ROUTE_RUN_CNT', 'VH_ID', 'BUS_RUN_CNT', 'STATION_ORD'])
#df.loc[:, 'EVENT_DATE'] = df['EVENT_DATE'].apply(lambda datetime: pd.to_datetime(datetime))

df = df.query('EVENT_CD in (17, 18)')

grouped = df.groupby(by = ['ROUTE_ID', 'ROUTE_RUN_CNT', 'VH_ID', 'BUS_RUN_CNT'])

result_file_path = infix_path + '_RUN_SPED_ANALYSIS' + postfix_path
result_cols = ['EVENT_DATE', 'EVENT_DATE2', 'STATION_ORD', \
               'STATION_ORD2', 'ROUTE_ID', 'RUN_DIST', 'RUN_SPED']
header = pd.DataFrame(columns = result_cols)
header.to_csv(result_file_path, index=False)
#del header

for name, group in grouped:
    sorted_group = group.sort_values(by = ['EVENT_DATE', 'EVENT_CD'], ascending = [True, True])
    
    # 버스 출발/도착 이벤트 로그가 중복되어 저장된 경우 -> 해당 로그 튜플 삭제
    # 예) EVENT_CD | STATION_ORD (2019.05.31 BIS, 노선ID 405130001, 6회차)
    #     ---------+------------
    #     18       | 1             -> 삭제 대상
    #     17       | 1             -> 삭제 대상
    #     18       | 1             -> 삭제 대상
    #     17       | 1             -> 삭제 대상
    #     18       | 1
    for station_ord, gr in sorted_group.groupby(by = 'STATION_ORD'):
        if len(gr) > 2:
            #print(name, '...', gr)
            # 이게 올바른 방법인지에 대한 추가적인 가치 판단이 필요하다.
            sorted_group = sorted_group.drop(gr.index[:-1])
    
    # 첫 행의 EVENT_CD가 18(출발)이 아닌 경우 -> 해당 로그 튜플 삭제
    if len(sorted_group) > 0 and \
            sorted_group.loc[sorted_group.index[0]]['EVENT_CD'] == 17:
        sorted_group = sorted_group.drop(sorted_group.index[0])
    
    # 마지막 행의 EVENT_CD가 17(도착)이 아닌 경우 -> 해당 로그 튜플 삭제
    if len(sorted_group) > 0 and \
            sorted_group.loc[sorted_group.index[-1]]['EVENT_CD'] == 18:
        sorted_group = sorted_group.drop(sorted_group.index[-1])
        
    # 출발/도착 결측 값 보정
    tup = (len(sorted_group[sorted_group['EVENT_CD'] == 18]), \
           len(sorted_group[sorted_group['EVENT_CD'] == 17]))
    if tup[0] != tup[1]:
        start, end = 0, len(sorted_group) - 1
        for i, _ in enumerate(sorted_group.groupby(by = 'STATION_ORD')):
            station_ord, gr = _[0], _[1]
            if i in (start, end): 
                continue
            
            if len(gr) == 1:
                temp = gr.loc[gr.index[0]]
                temp['EVENT_CD'] = 17 if gr.loc[gr.index[0]]['EVENT_CD'] == 18 else 18
                
                sorted_group.append(temp)
        
        sorted_group = sorted_group.sort_values(by = ['EVENT_DATE', 'EVENT_CD'], ascending = [True, True])
                
    # {18(출발) - 17(도착)} 테이블 병합
    cols = ['EVENT_DATE', 'RUN_DIST', 'STATION_ORD', 'ROUTE_ID']
    start_df = sorted_group.query('EVENT_CD == 18').reset_index()[cols]
    end_df = sorted_group.query('EVENT_CD == 17').reset_index()[cols]

    for col in cols:
        start_df[col + '2'] = end_df[col]

    # EVENT_DATE 타입 datetime64로 변환
    start_df.loc[:, 'EVENT_DATE_TEMP'] = \
        start_df['EVENT_DATE'].astype('datetime64')
        #start_df['EVENT_DATE'].apply(lambda datetime: pd.to_datetime(datetime))
    start_df.loc[:, 'EVENT_DATE_TEMP2'] = \
        start_df['EVENT_DATE2'].astype('datetime64')
        #start_df['EVENT_DATE2'].apply(lambda datetime: pd.to_datetime(datetime))

    # 구간별 운행거리 및 운행속도 계산
    # (구간별 운행 속도 m / 구간별 소요시간 s) * 3.6 -> km/h 단위로 변환.
    start_df['RUN_DIST'] = start_df['RUN_DIST2'] - start_df['RUN_DIST']
    start_df['RUN_SPED'] = (start_df['RUN_DIST'] / \
                            ((start_df['EVENT_DATE_TEMP2'].dt.hour*3600 + \
                              start_df['EVENT_DATE_TEMP2'].dt.minute*60 + \
                              start_df['EVENT_DATE_TEMP2'].dt.second) -
                             (start_df['EVENT_DATE_TEMP'].dt.hour*3600 + \
                              start_df['EVENT_DATE_TEMP'].dt.minute*60 + \
                              start_df['EVENT_DATE_TEMP'].dt.second))*3.6).round(1)

    start_df = start_df[result_cols]

    # csv 파일로 출력
    start_df.to_csv(result_file_path, index=False, header=False, mode='a')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
